# Imports

In [5]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [1]:
import cv2
import numpy as np

In [2]:
from matplotlib import pyplot as plt
from retinaface.pre_trained_models import get_model
from retinaface.utils import vis_annotations

In [3]:
import os
from tqdm import tqdm

# Functions & model

In [4]:
def is_specialfile(path,exts):
    _, file_extension = os.path.splitext(path)
    return file_extension.lower() in exts

img_extensions=['.jpg','.jpeg','.png']
def is_image(path):
    return is_specialfile(path,img_extensions)

In [6]:
from retinaface.pre_trained_models import get_model
model = get_model("resnet50_2020-07-20", max_size=1024, device='cuda')
model.eval()

/home/hse_student/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hse_student/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/hse_student/.local/lib/python3.8/site-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


In [7]:
import math
from skimage import transform as trans

def preprocess(img, bbox=None, landmark=None, **kwargs):
    M = None
    image_size = [224,224]
    src = np.array([
      [30.2946, 51.6963],
      [65.5318, 51.5014],
      [48.0252, 71.7366],
      [33.5493, 92.3655],
      [62.7299, 92.2041] ], dtype=np.float32 )
    if image_size[1]==224:
        src[:,0] += 8.0
    src*=2
    if landmark is not None:
        dst = landmark.astype(np.float32)

        tform = trans.SimilarityTransform()
        tform.estimate(dst, src)
        M = tform.params[0:2,:]

    if M is None:
        if bbox is None: #use center crop
            det = np.zeros(4, dtype=np.int32)
            det[0] = int(img.shape[1]*0.0625)
            det[1] = int(img.shape[0]*0.0625)
            det[2] = img.shape[1] - det[0]
            det[3] = img.shape[0] - det[1]
        else:
              det = bbox
        margin = kwargs.get('margin', 44)
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin//2, 0)
        bb[1] = np.maximum(det[1]-margin//2, 0)
        bb[2] = np.minimum(det[2]+margin//2, img.shape[1])
        bb[3] = np.minimum(det[3]+margin//2, img.shape[0])
        ret = img[bb[1]:bb[3],bb[0]:bb[2],:]
        if len(image_size)>0:
              ret = cv2.resize(ret, (image_size[1], image_size[0]))
        return ret 
    else: #do align using landmark
        assert len(image_size)==2
        warped = cv2.warpAffine(img,M,(image_size[1],image_size[0]), borderValue = 0.0)
        return warped

# Сreating destination folders

In [8]:
# example on CASIA-WebFace
path_dir = '/home/hse_student/apsidorova/dataset_mae/faces_webface_112x112/unbin_images'
outfile_cropped = 'CASIA-WebFace/cropped'
outfile_aligned = 'CASIA-WebFace/aligned'

In [9]:
from tqdm import tqdm

!mkdir CASIA-WebFace
!mkdir $outfile_cropped
!mkdir $outfile_aligned
for folder in os.listdir(path_dir):
    f = os.path.join(path_dir, folder)
    if os.path.isdir(f):
        !mkdir $outfile_cropped/$folder
        !mkdir $outfile_aligned/$folder

# Crop and Align images

In [ ]:
for folder in os.listdir(path_dir):
    f = os.path.join(path_dir, folder)
    print(f)
    if os.path.isdir(f):
        for image_name in tqdm(os.listdir(f)):
            path_image = os.path.join(f, image_name)
            if is_image(image_name):
                image = cv2.imread(path_image)
                annotations = model.predict_jsons(image)
                
                if not annotations[0]["bbox"]:
                    print('No faces')
                else:
                    for annotation in annotations:
                        box = np.array(annotation['bbox']).astype(int)
                        x1,y1,x2,y2=box[0:4]
                        x1=max(x1,0)
                        y1=max(y1,0)
                        face_img=image[y1:y2,x1:x2,:]
                        p=np.array(annotation['landmarks'])

                        cv2.imwrite(os.path.join(outfile_cropped, 
                                                 folder,
                                                 image_name), 
                                    face_img) 

                        face_img=preprocess(image,box,p)
                        cv2.imwrite(os.path.join(outfile_aligned, 
                                                 folder,
                                                 image_name), 
                                    face_img) 